In [ ]:
%run secscan-common.ipynb


In [ ]:
import json
epss_lookup = load_epss_map()
cve_src_map, cve_bin_map = cve_package_map()

In [ ]:
print(cve_src_map["CVE-2019-11840"])
print(cve_bin_map["CVE-2019-11840"])

In [ ]:
import os
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

# Initialize the LLM - using the correct model name from documentation
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash",  # Latest model from the docs
    api_key=GOOGLE_API_KEY,
    temperature=0.1
)

# Define system prompt for the agent
system_prompt = """You are a senior Debian Security Analyst. Your task is as follows:
1. Scan the system for vulnerabilities using the `scan_system_for_vulnerabilities` tool based on number of vulnerabilities input by user.
2. Pass the CVE List from the scan to the `research_vulnerabilities` tool to get detailed information.
3. Analyse the vulnerabilities and prepare a report summarising the top vulnerabilities and system status and remediation.
4. Your mission fails if you fail to execute both tools correctly

Always use both tools in sequence - first scan for vulnerabilities, then research the found CVEs."""

def extract_content(result):
    """Enhanced content extraction that works with different model response formats"""
    try:
        # Method 1: Direct content attribute (most common)
        if hasattr(result, 'content'):
            content = result.content
            # Handle list of content blocks (Gemini 3.x format)
            if isinstance(content, list):
                text_parts = []
                for block in content:
                    if isinstance(block, dict):
                        if block.get('type') == 'text':
                            text_parts.append(block.get('text', ''))
                        elif 'text' in block:
                            text_parts.append(block['text'])
                    else:
                        text_parts.append(str(block))
                return '\n'.join(text_parts) if text_parts else str(content)
            # Handle string content (Gemini 2.5 format)
            elif isinstance(content, str):
                return content
            else:
                return str(content)
        
        # Method 2: Dict with messages array (agent response format)
        elif isinstance(result, dict):
            if 'messages' in result and result['messages']:
                last_message = result['messages'][-1]
                return extract_content(last_message)
            elif 'content' in result:
                return extract_content(type('obj', (object,), {'content': result['content']})())
            elif 'output' in result:
                return str(result['output'])
            elif 'text' in result:
                return str(result['text'])
        
        # Method 3: AIMessage object with different attributes
        elif hasattr(result, 'text'):
            return result.text
        elif hasattr(result, 'message'):
            return extract_content(result.message)
        
        # Method 4: Fallback to string representation
        return str(result)
    
    except Exception as e:
        return f"Error extracting content: {e}\nRaw result: {str(result)}"

# Create the agent using the modern LangChain 1.x approach
agent = create_agent(
    model=llm,
    tools=[tool(scan_system_for_vulnerabilities), tool(research_vulnerabilities)],
    system_prompt=system_prompt
)

# Execute the security scan
try:
    result = agent.invoke({
        "messages": [{"role": "user", "content": "Scan my system for top 5 vulnerabilities and provide a comprehensive security analysis report"}]
    })
    
    print("\n" + "="*50)
    print("SECURITY ANALYSIS REPORT")
    print("="*50)
    
    # Extract content using enhanced method
    content = extract_content(result)
    print(content)
    
    # Debug info (commented out - uncomment if needed for troubleshooting)
    # print(f"\n[DEBUG] Result type: {type(result)}")
    # print(f"[DEBUG] Result attributes: {dir(result)}")
    # print(f"[DEBUG] Raw result: {result}")
        
except Exception as e:
    print(f"Error executing agent: {e}")
    print(f"Error type: {type(e)}")
    print("\nPlease check:")
    print("- GOOGLE_API_KEY environment variable is set")
    print("- debsecan is installed and accessible")
    print("- Network connectivity for CVE research")
    print("- LangChain dependencies are properly installed")

## Gemini 2.5 Flash output

### Debian Security Analysis Report

**Date:** October 26, 2024

**1. Introduction**
This report summarizes the findings of a security scan conducted on your system, identifying the top 5 vulnerabilities. The analysis includes detailed information on each vulnerability, its potential impact, and recommended remediation strategies.

**2. System Status**
The system currently has 5 identified vulnerabilities across `apt-cacher-ng`, `augeas`, and `avahi` packages. All identified vulnerabilities are currently awaiting a fix from the Debian security team, indicating that immediate patches are not yet available. While the Exploit Prediction Scoring System (EPSS) scores for these vulnerabilities are relatively low, indicating a lower likelihood of active exploitation, their presence still poses a risk to the system's integrity and availability.

**3. Top Vulnerabilities**

Here are the top 5 vulnerabilities identified on your system:

*   **CVE-2025-11147: Reflected Cross-Site Scripting (XSS) in Apt-Cacher-NG**
    *   **Affected Package:** `apt-cacher-ng`
    *   **Criticality:** CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N (Medium)
    *   **Details:** This vulnerability in Apt-Cacher-NG v3.2.1 allows for reflected cross-site scripting (XSS) attacks. Malicious scripts can be executed in `/html/<filename>.html`, potentially leading to session hijacking, defacement, or other client-side attacks if a user is tricked into clicking a malicious link.
    *   **Current Status:** AWAITING_FIX (No fix version assigned yet)
    *   **EPSS Score:** 0.00039 (Percentile: 0.11678) - Indicates a very low probability of exploitation in the wild.

*   **CVE-2025-2588: Null Pointer Dereference in Hercules Augeas**
    *   **Affected Package:** `augeas` (`augeas-lenses`, `augeas-tools`)
    *   **Criticality:** CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:N/I:N/A:L (Low)
    *   **Details:** A problematic vulnerability in Hercules Augeas 1.14.1 affects the `re_case_expand` function in `src/fa.c`. Manipulation of the `re` argument can lead to a null pointer dereference, potentially causing a denial of service. This vulnerability requires local access to exploit.
    *   **Current Status:** AWAITING_FIX (No fix version assigned yet)
    *   **EPSS Score:** 0.00177 (Percentile: 0.39384) - Indicates a low probability of exploitation in the wild.

*   **CVE-2024-52615: Fixed Source Ports in Avahi-daemon DNS Queries**
    *   **Affected Package:** `avahi` (`avahi-daemon`)
    *   **Criticality:** CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:L/A:N (Medium)
    *   **Details:** Avahi-daemon relies on fixed source ports for wide-area DNS queries. This design flaw simplifies the injection of malicious DNS responses by attackers, potentially leading to DNS spoofing and redirection to malicious sites.
    *   **Current Status:** AWAITING_FIX (No fix version assigned yet)
    *   **EPSS Score:** 0.00068 (Percentile: 0.21253) - Indicates a very low probability of exploitation in the wild.

*   **CVE-2024-52616: Predictable DNS Transaction IDs in Avahi-daemon**
    *   **Affected Package:** `avahi` (`avahi-daemon`)
    *   **Criticality:** CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:L/A:N (Medium)
    *   **Details:** Avahi-daemon initializes DNS transaction IDs randomly only once at startup and then increments them sequentially. This predictable behavior makes it easier for attackers to guess transaction IDs, facilitating DNS spoofing attacks.
    *   **Current Status:** AWAITING_FIX (No fix version assigned yet)
    *   **EPSS Score:** 0.00086 (Percentile: 0.25054) - Indicates a very low probability of exploitation in the wild.

**4. Remediation Recommendations**

Given that all identified vulnerabilities are currently awaiting a fix, the primary remediation strategy involves monitoring and timely application of updates.

*   **Monitor Debian Security Advisories:** Regularly check the Debian Security Advisories (DSAs) for updates related to `apt-cacher-ng`, `augeas`, and `avahi`. Subscribe to the debian-security-announce mailing list to receive immediate notifications when fixes are released.
*   **Apply Updates Promptly:** As soon as security updates are available for the affected packages, apply them to your system without delay. Use `sudo apt update && sudo apt upgrade` or `sudo apt-get dist-upgrade` to ensure all security patches are installed.
*   **Network Segmentation (for Avahi-daemon):** For CVE-2024-52615 and CVE-2024-52616, consider isolating systems running Avahi-daemon on a separate network segment or restricting its network access to only trusted hosts if its functionality is not required for wide-area DNS queries. This can limit the attack surface for DNS spoofing.
*   **User Awareness and Input Validation (for Apt-Cacher-NG):** For CVE-2025-11147, educate users about the risks of clicking suspicious links. While awaiting a fix, ensure that any user-supplied input to Apt-Cacher-NG is properly validated and sanitized to mitigate XSS risks.
*   **Least Privilege Principle:** Ensure that services and applications run with the minimum necessary privileges to reduce the impact of a successful exploit.

**5. Conclusion**

Your system has several vulnerabilities that require attention. While the EPSS scores suggest a lower immediate threat of exploitation, the presence of these flaws, especially those allowing network-based attacks (Avahi-daemon), necessitates proactive monitoring and remediation. It is crucial to stay informed about Debian security updates and apply them as soon as they become available to maintain a robust security posture. Continuous vigilance and adherence to security best practices are essential for protecting your system against evolving threats.

In [ ]:
#print(scan_system_status())
cve_pkg_map.get('CVE-2024-27280')

In [ ]:
from langchain.tools import tool
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

llm = ChatOllama(
    model="granite4",
    base_url="http://localhost:11434",
    temperature=0,
)

tools = [tool(scan_system_status)]
prompt = """You are a Senior Debian Security Analyst tasked with creating professional security assessment reports for executive stakeholders. Analyze ONLY the tool output data provided.

CRITICAL REQUIREMENTS:
- Use ONLY data from the tool output - no external knowledge or assumptions
- Create a comprehensive, executive-level security report
- Extract CVE ID, package name, CVSS score, EPSS score, and status from each vulnerability
- Provide detailed impact analysis based ONLY on the vulnerability descriptions provided
- Use these CVSS severity labels: 0.0-3.9 (Low), 4.0-6.9 (Medium), 7.0-8.9 (High), 9.0-10.0 (Critical)

REQUIRED REPORT FORMAT:

# Professional Security Assessment Report

## Executive Summary
[Brief overview: total vulnerabilities found, affected packages, overall risk assessment]

## System Status Overview  
[Concise summary of current security posture and key statistics]

## Critical Vulnerability Analysis
For EACH vulnerability from tool data, provide:

### CVE-[ID]: [Vulnerability Name from Description]
- **Package:** [from data]
- **Severity:** [CVSS score] ([Low/Medium/High/Critical label])  
- **Attack Vector:** [Extract from CVSS: Local/Network/Adjacent]
- **Technical Impact:** [Based on vulnerability description - be specific about what the flaw allows]
- **Business Risk:** [Translate technical impact to business consequences]
- **Exploit Probability:** [EPSS score and percentile from data]
- **Current Status:** [fix status from data]

## Risk Prioritization Matrix
[Rank vulnerabilities by combining CVSS score and EPSS probability]

## Detailed Remediation Strategy
### Immediate Actions (0-7 days):
[Specific actions based on vulnerability types found]

### Short-term Actions (1-4 weeks):
[Package-specific monitoring and preparation steps]

### Long-term Strategy:
[Ongoing security practices and monitoring]

## Conclusion and Next Steps
[Professional assessment of overall security posture and key action items]

TONE: Professional, authoritative, actionable. Write for security executives who need both technical depth and business context."""

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=prompt,
)

inputs = {
    "messages": [
        {
            "role": "user",
            "content": "Conduct a comprehensive security assessment of the top 5 vulnerabilities on my system. Provide an executive-level security report with detailed analysis and actionable remediation strategies."
        }
    ]
}

# Iterate through the stream
for chunk in agent.stream(inputs, stream_mode="updates"):
    for step_name, data in chunk.items():
        print(f"\n[Step: {step_name}]")
        
        # Pull the last message from this step's update
        last_msg = data["messages"][-1]
        
        # Use pretty_print() for a clean display of each step
        last_msg.pretty_print()

        # Access token usage metadata if it exists
        if hasattr(last_msg, 'usage_metadata') and last_msg.usage_metadata:
            usage = last_msg.usage_metadata
            print(f"\n--- Token Stats ---")
            print(f"Input Tokens: {usage.get('input_tokens')}")
            print(f"Output Tokens: {usage.get('output_tokens')}")
            print(f"Total Tokens: {usage.get('total_tokens')}")    

# llama3.1 analysis post refining prompt

## Professional Security Assessment Report

## Executive Summary
Total vulnerabilities found: 5  
Affected packages: grpc, shadow, bluez (2 instances)  
Overall risk assessment: High risk due to presence of critical and high-severity vulnerabilities.

## System Status Overview  
Current security posture: Vulnerable to remote code execution attacks via network-adjacent vectors.  
Key statistics: 4 out of 5 top vulnerabilities are awaiting fixes with no assigned version yet.

## Critical Vulnerability Analysis

### CVE-2023-44487
- **Package:** grpc
- **Severity:** CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H (High)
- **Attack Vector:** Network
- **Technical Impact:** Denial of service via server resource consumption.
- **Business Risk:** Potential for significant downtime and revenue loss due to unavailability of services.
- **Exploit Probability:** EPSS score 0.94427, percentile 0.99981
- **Current Status:** Awaiting fix with no assigned version yet.

### CVE-2024-56433
- **Package:** shadow
- **Severity:** CVSS:3.1/AV:L/AC:H/PR:L/UI:N/S:U/C:L/I:L/A:N (Medium)
- **Attack Vector:** Local
- **Technical Impact:** Potential account takeover via uid conflicts.
- **Business Risk:** Confidentiality and integrity risks due to unauthorized access.
- **Exploit Probability:** EPSS score 0.05074, percentile 0.89507
- **Current Status:** Awaiting fix with no assigned version yet.

### CVE-2023-51596
- **Package:** bluez
- **Severity:** CVSS:3.0/AV:A/AC:H/PR:N/UI:R/S:U/C:H/I:H/A:H (High)
- **Attack Vector:** Network-adjacent
- **Technical Impact:** Remote code execution via Bluetooth.
- **Business Risk:** Potential for data breaches and unauthorized access.
- **Exploit Probability:** EPSS score 0.02808, percentile 0.85742
- **Current Status:** Awaiting fix with no assigned version yet.

### CVE-2019-11840
- **Package:** None (system impact only)
- **Severity:** CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:N/A:N (High)
- **Attack Vector:** Network
- **Technical Impact:** Potential for keystream attacks.
- **Business Risk:** Confidentiality risks due to unauthorized access.
- **Exploit Probability:** EPSS score 0.02757, percentile 0.85625
- **Current Status:** No packages affected on this Debian version.

### CVE-2023-44431
- **Package:** bluez
- **Severity:** CVSS:3.1/AV:A/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H (High)
- **Attack Vector:** Network-adjacent
- **Technical Impact:** Remote code execution via Bluetooth.
- **Business Risk:** Potential for data breaches and unauthorized access.
- **Exploit Probability:** EPSS score 0.02464, percentile 0.84867
- **Current Status:** Awaiting fix with no assigned version yet.

## Risk Prioritization Matrix

| CVE ID | CVSS Score | EPSS Score | Priority |
| --- | --- | --- | --- |
| DEBIAN-CVE-2023-44487 | 9.0 | 0.94427 | High |
| DEBIAN-CVE-2024-56433 | 6.5 | 0.05074 | Medium |
| DEBIAN-CVE-2023-51596 | 8.8 | 0.02808 | High |
| DEBIAN-CVE-2019-11840 | 7.5 | 0.02757 | High |
| DEBIAN-CVE-2023-44431 | 8.8 | 0.02464 | High |

## Detailed Remediation Strategy

### Immediate Actions (0-7 days):
1. Apply temporary mitigations for CVE-2023-51596 and CVE-2023-44487 to prevent remote code execution attacks.
2. Implement additional security measures, such as network segmentation and access controls, to minimize potential damage.

### Short-term Actions (1-4 weeks):
1. Monitor package updates for grpc, shadow, and bluez to ensure timely installation of fixes.
2. Conduct regular vulnerability scans to detect any newly introduced vulnerabilities.

### Long-term Strategy:
1. Implement a robust patch management process to ensure timely installation of security patches.
2. Regularly review and update security policies and procedures to reflect changing threat landscapes.
3. Provide ongoing training and awareness programs for system administrators and users on best practices for secure configuration and usage.

## Conclusion and Next Steps
The current security posture is vulnerable to remote code execution attacks via network-adjacent vectors. Immediate action is required to apply temporary mitigations and implement additional security measures. Short-term actions include monitoring package updates and conducting regular vulnerability scans. Long-term strategy involves implementing a robust patch management process, reviewing and updating security policies, and providing ongoing training and awareness programs.


## LLama needs refined prompts for better answer
- Llama models respond well to very explicit structure requirements
- Breaking down complex tasks into specific sub-requirements improves output
- Providing context about the audience (executives) helps tone and detail level
- Explicit instructions about combining technical and business perspectives

# Granite4 3.4B Parameters Model output

## Executive Summary
The security assessment of the top five vulnerabilities on the system reveals a critical exposure due to unresolved issues in the **grpc**, **shadow-utils**, and **bluez** packages, each with high CVSS scores indicating severe risk levels. The presence of these critical vulnerabilities underscores an immediate need for remediation to prevent potential exploitation.

## System Status Overview  
The current security posture shows five critical vulnerabilities awaiting fixes across key system components: grpc (HTTP/2 protocol denial-of-service), shadow-utils (default subuid settings), and bluez (multiple remote code execution flaws). No other packages are affected by these issues, highlighting a focused risk concentration on network services and Bluetooth functionality.

## Critical Vulnerability Analysis

### CVE-DEBIAN-CVE-2023-44487
- **Package:** grpc  
- **Severity:** CVSS 3.1 (High)  
- **Attack Vector:** Network (Network)  
- **Technical Impact:** Allows denial of service by exploiting request cancellation to rapidly consume server resources, observed in the wild from August through October 2023.  
- **Business Risk:** Potential disruption of services leading to downtime and loss of revenue; reputational damage if sensitive data is compromised during resource exhaustion.  
- **Exploit Probability:** EPSS Score: 0.94394 (99.97th percentile), indicating a very high likelihood of exploitation in the wild.  
- **Current Status:** AWAITING_FIX  

### CVE-DEBIAN-CVE-2024-56433
- **Package:** shadow-utils  
- **Severity:** CVSS 3.1 (High)  
- **Attack Vector:** Local (Local)  
- **Technical Impact:** Default /etc/subuid settings can conflict with user uids on locally administered networks, potentially leading to account takeover by leveraging the default configuration.  
- **Business Risk:** Unauthorized access to critical systems and data, increasing risk of further exploitation or lateral movement within the network.  
- **Exploit Probability:** EPSS Score: 0.05074 (89.5th percentile), indicating a high likelihood of exploitation given the realistic conflict scenarios in local environments.  
- **Current Status:** AWAITING_FIX  

### CVE-DEBIAN-CVE-2023-51596
- **Package:** bluez  
- **Severity:** CVSS 3.0 (High)  
- **Attack Vector:** Adjacent Network (Network)  
- **Technical Impact:** Heap-based buffer overflow vulnerability in the Phone Book Access Profile, enabling remote code execution on affected installations via Bluetooth connections.  
- **Business Risk:** Full compromise of device functionality and data integrity; potential for unauthorized access to sensitive information transmitted over Bluetooth.  
- **Exploit Probability:** EPSS Score: 0.02808 (85.75th percentile), indicating a high probability of exploitation in environments where Bluetooth services are active.  
- **Current Status:** AWAITING_FIX  

### CVE-DEBIAN-CVE-2019-11840
- **Package:** golang.org/x/crypto (Go cryptography library)  
- **Severity:** CVSS 3.1 (High)  
- **Attack Vector:** Network (Network)  
- **Technical Impact:** Flaw in the amd64 implementation of Salsa20 and related packages leading to remote code execution if more than 256 GiB of keystream data is processed.  
- **Business Risk:** Potential compromise of cryptographic operations, impacting confidentiality and integrity of encrypted communications.  
- **Exploit Probability:** EPSS Score: 0.02757 (85.63th percentile), indicating a high likelihood of exploitation in environments utilizing the affected Go packages for encryption tasks.  
- **Current Status:** No Packages Affected  

### CVE-DEBIAN-CVE-2023-44431
- **Package:** bluez  
- **Severity:** CVSS 3.1 (High)  
- **Attack Vector:** Adjacent Network (Network)  
- **Technical Impact:** AVRCP Stack-based buffer overflow vulnerability enabling remote code execution via Bluetooth connections, requiring user interaction to exploit.  
- **Business Risk:** Full compromise of device control and data integrity; potential for unauthorized access to sensitive information transmitted over Bluetooth.  
- **Exploit Probability:** EPSS Score: 0.02464 (84.87th percentile), indicating a high likelihood of exploitation given the realistic attack scenarios in Bluetooth environments.  
- **Current Status:** AWAITING_FIX  

## Risk Prioritization Matrix
| CVE ID            | CVSS Score | EPSS Score | Priority |
|-------------------|------------|------------|----------|
| DEBIAN-CVE-2023-44487 | 3.1        | 0.94394    | Critical |
| DEBIAN-CVE-2024-56433 | 3.1        | 0.05074    | High     |
| CVE-DEBIAN-CVE-2023-51596 | 3.0      | 0.02808    | Critical |
| CVE-DEBIAN-CVE-2019-11840 | 3.1       | 0.02757   | High     |
| DEBIAN-CVE-2023-44431 | 3.1        | 0.02464    | Critical |

## Detailed Remediation Strategy

### Immediate Actions (0-7 days)
- **Patch Deployment:** Prioritize deploying patches for grpc, shadow-utils, and bluez to mitigate high-risk vulnerabilities.
- **Network Segmentation:** Implement strict network segmentation controls to limit lateral movement in case of successful exploitation.

### Short-term Actions (1-4 weeks)
- **Package Monitoring:** Enable real-time monitoring for grpc, shadow-utils, and bluez packages to detect any anomalous activity post-patch deployment.
- **User Education:** Conduct training sessions on recognizing phishing attempts or suspicious Bluetooth connections that could exploit these vulnerabilities.

### Long-term Strategy
- **Ongoing Security Practices:** Establish a continuous vulnerability scanning schedule to identify future issues before they can be exploited.
- **Incident Response Plan Update:** Revise the incident response plan to include procedures for rapid patch deployment and containment of potential breaches involving grpc, shadow-utils, or bluez vulnerabilities.

## Conclusion and Next Steps
The current security posture is significantly compromised by unresolved critical vulnerabilities in key system components. Immediate action is required to deploy patches for grpc (3.1), shadow-utils (3.1), and bluez (3.0) to prevent exploitation. Concurrently, implement network segmentation controls and enhance user awareness training to reduce the risk of successful attacks. Ongoing monitoring and regular security assessments are essential to maintain a robust defense against emerging threats.

**Next Steps:**  
- Initiate patch deployment for all identified vulnerabilities within the next 7 days.
- Conduct a comprehensive review of system configurations, particularly those related to grpc and Bluetooth services, to ensure no residual risks remain.
- Schedule a follow-up assessment in four weeks to evaluate the effectiveness of remediation efforts and adjust strategies as necessary.
